In [ ]:
# df = num_rowd.copy()
# df['year'] = df.recordtime.dt.year
# df['quarter'] = df.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int)
# df['monthly'] = df.recordtime.dt.strftime('%B')
# df['weekly'] = df.recordtime.dt.strftime('%W').astype(int)
# df['daily'] = df.recordtime.dt.date

# df.head(10)

In [1]:
import numpy as np
import pandas as pd
import dataset as ds

from datetime import datetime

In [2]:
num_rowd = ds.dbNumCrowd()

In [3]:
def filter_data(data, store, date = None, year = None, week = None, month = None, quarter = None):
    if date:
        data = data[data.recordtime.dt.date == date]
    else:
        data = data[data.recordtime.dt.year == year]
        if isinstance(week, int):
            data = data[data.recordtime.dt.strftime('%W').astype(int) == week]
        elif month:
            data = data[data.recordtime.dt.strftime('%B') == month]
        elif isinstance(quarter, int):
            data = data[data.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int) == quarter + 1]
    
    if store > 0:
        data = data[data.storeid == store]

    return data.sort_values(by = 'recordtime', ascending = True).reset_index(drop = True)

In [4]:
def set_variables(df, store_selected = 0, option_selected = 'Daily'):
    data = df.copy()
    date_selected, week_selected, month_selected, quarter_selected, year_selected = None, None, None, None, None

    if option_selected == 'Daily':
        date_selected = datetime(2022, 6, 2).date()
    else:
        year_selected = 2023
        if option_selected == 'Weekly':
            week_selected = 2
        elif option_selected == 'Monthly':
            month_selected = 'January'
        elif option_selected == 'Quarter':
            quarter_selected = 0

    return data, store_selected, date_selected, week_selected, month_selected, quarter_selected, year_selected


In [36]:
def clean_data(data, option, period = None):
    df = data.copy()

    df.drop(columns = ['position', 'storeid'], axis = 1, inplace = True)

    df['in_num'] = df.in_num.where(df.in_num < 200, df.in_num * 0.01).apply(np.int64)
    df['out_num'] = df.out_num.where(df.out_num < 200, df.out_num * 0.01).apply(np.int64)

    if option == 'Daily':
        freqs = ['5min', '15min', '30min', 'H']
        df = df.resample(freqs[period], on = 'recordtime').sum()
    elif option == 'Weekly':
        df = df.resample('D', on = 'recordtime').sum().reset_index()
        df['day_name'] = df.recordtime.dt.day_name()
        df.set_index(['recordtime', 'day_name'], inplace = True)
    elif option == 'Monthly':
        df = df.resample('D', on = 'recordtime').sum()#.reset_index()
    else:
        df = df.resample('M', on = 'recordtime').sum().reset_index()
        df['recordtime'] = df.recordtime.dt.strftime('%Y-%m')
        df.set_index('recordtime', inplace = True)
    
    return df

In [45]:
option = ['Daily', 'Weekly', 'Monthly', 'Quarter', 'Yearly']
option_selected = option[4]

variables = set_variables(num_rowd.copy(), option_selected = option_selected)

data = variables[0]
store_selected = variables[1]
date_selected = variables[2]
week_selected = variables[3]
month_selected = variables[4]
quarter_selected = variables[5]
year_selected = variables[6]

df = filter_data(data, store_selected, date_selected, year_selected, week_selected, month_selected, quarter_selected)

In [46]:
period_selected = 3

df_1 = clean_data(df, option_selected, period_selected)
df_1

,in_num,out_num
recordtime,,
2023-01,582462,584679
2023-02,695029,710882
2023-03,984057,1028978
2023-04,5772695,6165439


In [ ]:
# d = datetime(2023, 3, 2).date()

pd.date_range(datetime.combine(datetime(2023, 3, 21), datetime.min.time()), 
              datetime.combine(datetime(2023, 3, 21), datetime.max.time()), freq = '15min')

# https://www.appsloveworld.com/pandas/100/54/sum-based-on-date-range-in-two-separate-columns